In [1]:
import sh
import json
import os
import sys
import re
import io
import pandas as pd
import subprocess

In [2]:
project_path = os.getcwd()
print(project_path)
project = "lucene-solr"
issues = "issue_LUCENE"
ISSUE_ID = "LUCENE"
repository = "https://github.com/apache/lucene-solr.git"

/Users/kajdreef/Documents/Programmeren/Master/IN4334-Mining_Software_repositories/dataset_creation/lucene


In [3]:
def print_progress(i, l, step=1):
    if i % step == 0:
            print("\r>> {}/{}".format(i + 1, l), end="")
            sys.stdout.flush()

In [4]:
if not os.path.exists(os.getcwd() + '/' + project):
    print("Path doesn\'t exists, cloning repo:")
    sh.git.clone(repository)
else:
    print("Repository exists")

Repository exists


In [6]:
git = sh.git.bake(_cwd=project)
git.checkout("trunk")
git.status()

On branch trunk
Your branch is behind 'origin/trunk' by 113 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
nothing to commit, working directory clean

In [7]:
def extract_bug(file):
    if not file.endswith('.json'):
            return None
    else:
        bug_json_string = open(issues + '/' + file).read()
        bug = json.loads(bug_json_string)
        bug_fields = bug.get('fields')
        
        if bug_fields['issuetype']['name'] != 'Bug':
            return None
   
        if bug_fields['resolution'] == None:
            return None 
                
        if bug_fields['resolution']['name'] != 'Fixed':
            return None
    return bug

In [8]:
#Extracting all the fixed bugs from the bug repo
bugs = dict() #Dictionary of bugs indexed by LUCENE+ID
dir_list = os.listdir(os.getcwd() + '/' + issues)
l = len(dir_list)
print("Progress:")
for idx, file in enumerate(dir_list):
    print_progress(idx,l,10)
    bug = extract_bug(file)
    if(bug == None):
        continue
    bugs[bug['key'].upper()] = bug

Progress:
>> 6641/6641

In [9]:
commits = []

log = git.log("--reverse", "--pretty=format:%H")
for c_hash in log:
    if(c_hash.endswith('\n')):
        c_hash = c_hash[:-1]
    commits.append(c_hash)
len(commits)

16066

In [10]:
def get_fixed_bugs(commit_hash):
    details = git.show("--name-only","--pretty=format:%B", commit_hash)
    #init = details.split(':')[0].upper()
    f_keys = []
    
    keys = re.findall(ISSUE_ID + "-[0-9]+",str(details))
    if len(keys) != 0:
        for k in keys:
            if bugs.get(k) != None:
                #Then it fixes a bug
                f_keys.append(k)
    return f_keys

In [11]:
#Splits the bug fixing ones from the others

fix_commits = [] #List that will be filled with couples (commit, bugs-fixed)
non_fix_commits = [] #List that will be filled only with commit hashes
l = len(commits)
print("Progress:")
for idx, c in enumerate(commits):
    
    print_progress(idx, l, 10)
        
    details = git.show("--name-only","--pretty=format:%B", c)
    files = git.show("--name-only","--pretty=format:",c)
    
    
    f_keys = get_fixed_bugs(c)    
    if len(f_keys) != 0:
        fix_commits.append((c,f_keys))
    else:
        non_fix_commits.append(c)
        
#Some commits cite more than one issue...which lines are the bug fix, if one of the two is only an improvement ?
#HP: we consider all the removed lines as implicated (fix-inducing)

Progress:
>> 16061/16066

In [12]:
print(len(fix_commits)) #Number of LUCENE bug fixing commits
print(len(non_fix_commits)) #Number of LUCENE non bug fixing commits
print(len(commits)) #Number of LUCENE + SOLR commits
#We can see that on average half of the commits were only solr related, so we discarded them

1836
14230
16066


In [19]:
def get_added_lines(commitHash):
    
    # Get diff of current and previous commit and store this in a file
    with open('diff_file.txt', 'wb+') as file:
        cmd = ["git " + "-C ./" + project + " diff --no-color " + commitHash +'^ ' + commitHash]
        gitDiff = subprocess.Popen(cmd, shell=True, universal_newlines=True, stdout=file)
        gitDiff.wait()
        file.flush()

    # Get diff of the commit
    output_diff = ""
    with open('diff_file.txt', 'rb+') as file:
        output_diff = file.read()
    
    file.close()
    
    # Cut output up in separate lines
    lines = output_diff.decode("latin1").split('\n')
    
    line_num = dict()
    start_line = 0
    name_of_file = ""
    i = 0
    name_file_set = set()

    # Go through the diff output to determine line number(s) of deleted lines
    for line in lines:
        # Header where the name of the file is located.
        if line[0:4] == "diff":
            name_of_file = line.split(' b/')[-1]


        # New section of changed code find starting line number
        elif line[0:2] == "@@":
            lineNum = re.findall('\-[0-9]+\,',line)
            start_line = int(re.findall('\-[0-9]+',line)[0][1:])
            i = 0

        # Lines that has been changed
        elif (line[0:2] == "- "):
            if line_num.get(name_of_file) == None:
                # create set of lines that have been changed for each file
                line_num[name_of_file] = []
                name_file_set.add(name_of_file)
            # Add lines to the set that have been changed
            line_num[name_of_file].append(start_line + i);
            i += 1


        # Indentaion infront of a line of code that hasn't changed
        elif line[0:2] == "  ":
            i += 1
            
    return line_num

In [20]:
def get_bug_commits(commit_hash, line_num, bug_hash):
    # checkout previous commit
    git.checkout("--force", commit_hash + "^")

    # Go through all the files that have been changed
    for key_file_name in line_num:
        ## Check if file is a java file if not continue with next file
        if(key_file_name.find('.java') == -1):
            continue
        
        # blame file to see when the lines where last changed (on previous commit)
        with open('blame_file.txt', 'wb+') as file:
            # --root is added to see boundary commits as normal commits
            cmd = ["git " + "-C ./"+ project +" blame -l --root " + key_file_name]
            gitBlame = subprocess.Popen(cmd, shell=True, universal_newlines=True, stdout=file)
            gitBlame.wait()
            file.flush()
            
        output_blame = ""
        with open('blame_file.txt', 'rb+') as file:
            output_blame = file.read()

        # Cut output up in separate lines
        blame_output = output_blame.decode("latin1")        
        blame_output_split = blame_output.split('\n')
                
        # Get hash for all the lines that have been changed.
        for i in line_num[key_file_name]:
            line = blame_output_split[i-1]
            hash_of_line = line.split(' ')[0]
            
            if bug_hash.get(hash_of_line) == None:
                bug_hash[hash_of_line] = set([key_file_name])
            else:
                bug_hash[hash_of_line].add(key_file_name)
            
    return bug_hash

In [21]:
total = len(fix_commits)
bug_commits = dict()

print("Progress:")
for idx, fix_hash in enumerate(fix_commits):
    new_lines = get_added_lines(fix_hash[0])
    bug_commits = get_bug_commits(fix_hash[0], new_lines, bug_commits)

    print_progress(idx, total,1)

Progress:
>> 1836/1836

In [22]:
hash_file_list = []
for key, file_name_set in bug_commits.items():
    for file_name in file_name_set:
        hash_file_list.append([key, file_name, 1])

In [23]:
df = pd.DataFrame(hash_file_list, columns=['commit_hash','file', 'implicated'])
df.to_csv("dataset/implicated_files.csv")

df


,commit_hash,file,implicated
0,43871e2fa652811188039e1176aaa7c3e1db0ccf,solr/core/src/java/org/apache/solr/response/PH...,1
1,8338fdc9af30b0e542d6cd6edfd8daa703f5686d,lucene/core/src/java/org/apache/lucene/store/N...,1
2,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/test/org/apache/solr/core/TestSo...,1
3,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/java/org/apache/solr/core/Config...,1
4,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/java/org/apache/solr/core/CoreCo...,1
5,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/java/org/apache/solr/cloud/ZkCon...,1
6,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/java/org/apache/solr/cloud/ZkCLI...,1
7,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/java/org/apache/solr/core/Config...,1
8,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/java/org/apache/solr/core/CoreDe...,1
9,4efc762f4eb5515b865f1827c6e794f20f7e69cb,solr/core/src/test/org/apache/solr/core/TestLa...,1


In [ ]:
new_lines